In [45]:
import pandas as pd

## 1 Preprocess User info data

#### 1.1 load user
load user and filter out users with no country info

In [46]:
# load user info
usr_info_df = pd.read_csv("user.csv")
user_df = usr_info_df.dropna(subset=['county'])
#print(usr_info_filterd_df)

#### 1.2 split user to different node

In [47]:
asia_countries = {
    "CN": "中国",
    "IN": "印度",
    "JP": "日本",
    "KR": "韩国",
    "ID": "印度尼西亚",
    "VN": "越南",
    "TH": "泰国",
    "PH": "菲律宾",
    "BD": "孟加拉国",
    "PK": "巴基斯坦",
    "IR": "伊朗",
    "TR": "土耳其",
    "SA": "沙特阿拉伯",
    "IQ": "伊拉克",
    "AE": "阿联酋",
    "AF": "阿富汗",
    "IL": "以色列",
    "LB": "黎巴嫩",
    "JO": "约旦",
    "SY": "叙利亚",
    "YE": "也门",
    "MN": "蒙古",
    "MY": "马来西亚",
    "SG": "新加坡",
    "KP": "朝鲜",
    "UZ": "乌兹别克斯坦",
    "KZ": "哈萨克斯坦",
    "KG": "吉尔吉斯斯坦",
    "TJ": "塔吉克斯坦",
    "TM": "土库曼斯坦",
    "AZ": "阿塞拜疆",
    "GE": "格鲁吉亚",
    "AM": "亚美尼亚",
    "LK": "斯里兰卡",
    "BT": "不丹",
    "NP": "尼泊尔",
    "MV": "马尔代夫",
}

asia_country_codes = list(asia_countries.keys())

europe_countries = {
    "GB": "英国",
    "DE": "德国",
    "FR": "法国",
    "ES": "西班牙",
    "IT": "意大利",
    "RU": "俄罗斯",
    "NL": "荷兰",
    "CH": "瑞士",
    "AT": "奥地利",
    "BE": "比利时",
    "SE": "瑞典",
    "NO": "挪威",
    "DK": "丹麦",
    "FI": "芬兰",
    "GR": "希腊",
    "PT": "葡萄牙",
    "IE": "爱尔兰",
    "UA": "乌克兰",
    "PL": "波兰",
    "CZ": "捷克共和国",
    "HU": "匈牙利",
    "RO": "罗马尼亚",
    "BY": "白俄罗斯",
    "SK": "斯洛伐克",
    "HR": "克罗地亚",
    "LT": "立陶宛",
    "SI": "斯洛文尼亚",
    "LV": "拉脱维亚",
    "EE": "爱沙尼亚",
    "ME": "黑山",
    "MK": "马其顿",
    "RS": "塞尔维亚",
    "AL": "阿尔巴尼亚",
    "BA": "波斯尼亚和黑塞哥维那",
    "XK": "克什米尔",
    "MD": "摩尔多瓦",
    "MT": "马耳他",
    "IS": "冰岛",
    "LI": "列支敦士登",
    "LU": "卢森堡",
    "AD": "安道尔",
    "MC": "摩纳哥",
    "SM": "圣马力诺",
    "VA": "梵蒂冈",
}

europe_country_codes = list(europe_countries.keys())
print(europe_country_codes)

africa_countries = {
    "NG": "尼日利亚",
    "EG": "埃及",
    "CM": "喀麦隆",
    "KE": "肯尼亚",
    "ZA": "南非",
    "GH": "加纳",
    "CI": "科特迪瓦",
    "MG": "马达加斯加",
    "MA": "摩洛哥",
    "SN": "塞内加尔",
    "DZ": "阿尔及利亚",
    "AO": "安哥拉",
    "ET": "埃塞俄比亚",
    "TZ": "坦桑尼亚",
    "LY": "利比亚",
    "CD": "刚果（金）",
    "TN": "突尼斯",
    "MZ": "莫桑比克",
    "SO": "索马里",
    "SL": "塞拉利昂",
    "UG": "乌干达",
    "ML": "马里",
    "NE": "尼日尔",
    "RW": "卢旺达",
    "TD": "乍得",
    "LR": "利比里亚",
    "MW": "马拉维",
    "ZM": "赞比亚",
    "DJ": "吉布提",
    "MR": "毛里塔尼亚",
    "GN": "几内亚",
    "GM": "冈比亚",
    "BI": "布隆迪",
    "GQ": "赤道几内亚",
    "ER": "厄立特里亚",
    "CV": "佛得角",
    "SC": "塞舌尔",
    "ST": "圣多美和普林西比",
    "KM": "科摩罗",
    "LS": "莱索托",
    "SZ": "斯威士兰",
    "MU": "毛里求斯",
}

africa_country_codes = list(africa_countries.keys())
print(africa_country_codes)

north_america_countries = {
    "US": "美国",
    "CA": "加拿大",
    "MX": "墨西哥",
    "CU": "古巴",
}

north_america_country_codes = list(north_america_countries.keys())
print(north_america_country_codes)

south_america_countries = {
    "BR": "巴西",
    "AR": "阿根廷",
    "CL": "智利",
    "CO": "哥伦比亚",
    "PE": "秘鲁",
    "VE": "委内瑞拉",
    "EC": "厄瓜多尔",
    "BO": "玻利维亚",
    "PY": "巴拉圭",
    "UY": "乌拉圭",
    "GY": "圭亚那",
    "SR": "苏里南",
}

south_america_country_codes = list(south_america_countries.keys())
print(south_america_country_codes)


oceania_countries = {
    "AU": "澳大利亚",
    "NZ": "新西兰",
    "FJ": "斐济",
    "PG": "巴布亚新几内亚",
    "SB": "所罗门群岛",
    "VU": "瓦努阿图",
    "TO": "汤加",
    "WS": "萨摩亚",
    "KI": "基里巴斯",
    "FM": "密克罗尼西亚联邦",
    "MH": "马绍尔群岛",
    "PW": "帕劳",
    "NR": "瑙鲁",
    "TV": "图瓦卢",
}

oceania_country_codes = list(oceania_countries.keys())
print(oceania_country_codes)

node_asia_country =  asia_country_codes + oceania_country_codes
node_europe_country = europe_country_codes + africa_country_codes
node_america_country = north_america_country_codes
node_latin_country = south_america_countries

subset1 = user_df[user_df['county'].isin(node_asia_country)].reset_index(drop=True)
subset2 = user_df[user_df['county'].isin(node_europe_country)].reset_index(drop=True)
subset3 = user_df[user_df['county'].isin(node_america_country)].reset_index(drop=True)
subset4 = user_df[user_df['county'].isin(node_latin_country)].reset_index(drop=True)

#print(subset1)
#print(subset2)
#print(subset3)
#print(subset4)

subset1.to_csv("node-asia-users.csv")
subset2.to_csv("node-europe-users.csv")
subset3.to_csv("node-america-users.csv")
subset4.to_csv("node-latin-users.csv")

['GB', 'DE', 'FR', 'ES', 'IT', 'RU', 'NL', 'CH', 'AT', 'BE', 'SE', 'NO', 'DK', 'FI', 'GR', 'PT', 'IE', 'UA', 'PL', 'CZ', 'HU', 'RO', 'BY', 'SK', 'HR', 'LT', 'SI', 'LV', 'EE', 'ME', 'MK', 'RS', 'AL', 'BA', 'XK', 'MD', 'MT', 'IS', 'LI', 'LU', 'AD', 'MC', 'SM', 'VA']
['NG', 'EG', 'CM', 'KE', 'ZA', 'GH', 'CI', 'MG', 'MA', 'SN', 'DZ', 'AO', 'ET', 'TZ', 'LY', 'CD', 'TN', 'MZ', 'SO', 'SL', 'UG', 'ML', 'NE', 'RW', 'TD', 'LR', 'MW', 'ZM', 'DJ', 'MR', 'GN', 'GM', 'BI', 'GQ', 'ER', 'CV', 'SC', 'ST', 'KM', 'LS', 'SZ', 'MU']
['US', 'CA', 'MX', 'CU']
['BR', 'AR', 'CL', 'CO', 'PE', 'VE', 'EC', 'BO', 'PY', 'UY', 'GY', 'SR']
['AU', 'NZ', 'FJ', 'PG', 'SB', 'VU', 'TO', 'WS', 'KI', 'FM', 'MH', 'PW', 'NR', 'TV']


## 2 Extract node trace

#### 2.1 load traces file

In [48]:
traces_df = pd.read_csv("traces.csv")
#print(traces_df)

# normalize timestamp
start_time = traces_df.loc[0, "timestamp"]
traces_df["time_offset"] = traces_df["timestamp"] - start_time
#traces_df["time_offset_seconds"] = traces_df["time_offset"].dt.total_seconds()
print(traces_df)

         user_id   timestamp method            file_url  file_size  n_like  \
0        2729230  1342124615   POST   2729230-38105.dat      87243       0   
1         312247  1343344074    GET   2729230-38105.dat      87243       0   
2        2141252  1348261916   POST  2141252-311106.dat      35829       0   
3         807036  1351263903   POST    807036-79676.dat      70419       0   
4        1294201  1354231696   POST  1294201-707381.dat      15008       0   
...          ...         ...    ...                 ...        ...     ...   
1514251  2632606  1374116655   POST   2632606-91913.dat      39779       0   
1514252   655898  1374116837   POST   655898-745271.dat      16330       0   
1514253   373736  1374117563   POST   373736-745339.dat     148238       0   
1514254  1226119  1374121165   POST  1226119-745438.dat      96134       0   
1514255  3003249  1374124945   POST  3003249-116806.dat      31237       0   

         time_offset  
0                  0  
1            1219

#### 2.2 split to each node

In [49]:
node_aisa_merged_traces = pd.merge(traces_df, subset1[['user_id']], on='user_id', how='inner')
node_europe_merged_traces = pd.merge(traces_df, subset2[['user_id']], on='user_id', how='inner')
node_america_merged_traces = pd.merge(traces_df, subset3[['user_id']], on='user_id', how='inner')
node_latin_merged_traces = pd.merge(traces_df, subset4[['user_id']], on='user_id', how='inner')

In [50]:
# export dataset
node_aisa_merged_traces.to_csv("node-aisa-traces.csv")
node_europe_merged_traces.to_csv("node-europe-traces.csv")
node_america_merged_traces.to_csv("node-america-traces.csv")
node_latin_merged_traces.to_csv("node-latin-traces.csv")